### Beschrijving

1. Koppeltabellen inlezen
2. Per bron de informatie toevoegen aan de koppeltabellen op het betreffende aggregatieniveau

Waarschuwing<br>
- CBS kerncijfers (ODATA) is nu de versie van 2019 maar wijzigt elk jaar, alsmede buurten, wijken en gemeenten, adressen<br>
- De code in dit programma moet achter elkaar gedraaid worden en geen stukjes apart

Input<br>
- Leefbaarometer, https://data.overheid.nl/dataset/leefbaarometer-meting-2018<br>
- CBS: buurten, wijken, gemeenten: Drie kleinere bestanden op https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer<br>
- CBS: adressen koppelbestand: https://www.cbs.nl/nl-nl/maatwerk/2019/42/buurt-wijk-en-gemeente-2019-voor-postcode-huisnummer <br>
- CBS: kerncijfers wijken en buurten, zowel in CBSODATA als hier: https://www.cbs.nl/nl-nl/maatwerk/2019/31/kerncijfers-wijken-en-buurten-2019
- CBS: nabijheidsstatistieken over voorzieningen in buurten, zowel in CBSODATA als hier: https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/wijk-en-buurtstatistieken/nabijheidsstatistieken-per-wijk-buurt
- CBS; maatwerktabel vernummering ontvangen van regstat@cbs.nl; betreft oude en nieuwe buurtcodes, ook af te leiden uit kerncijfers wijken en buurten adhv wijzigingsindicator
- Vesteda interne data: MobilityLabel: bereikbaarheidstabel per PC5 en markpotentie per gemeente<br>
- Calcasa: woningmarktinformatie per wijk, Vesteda data<br>
- Calcasa: woningvoorraad incl bouwjaren informatie per buurt, Vesteda data<br>
- ABF: data over midden en hoge inkomens per buurt, Vesteda data<br>

Output
- bm = buurten main: Bestand met alle buurten van Nederland en informatie daarover; als input voor analyses van buurten gezamenlijk met wm en gm
- wm = wijken main
- gm = gemeenten main
- buurten: bm,wm,gm samengevoegd

### Declaraties

In [2]:
# importeren packages
# package odata installeren: pip install cbsodata (zie ook https://pypi.org/project/cbsodata/)
import cbsodata
import pandas as pd
import numpy as np

# setten source path: later moet alle brondata naar datalake
source_path ="Q:\\RESEARCH\\RESEARCH ALGEMEEN\\Aantrekkelijkheid buurten\\DS project 2020\\Brondata\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "Q:\RESEARCH\RESEARCH ALGEMEEN\\Aantrekkelijkheid buurten\\DS project 2020\\Resultaten\\"

# jaar waarin gemeente op 1 januari een bevolking moet hebben om meegenomen te worden 
peiljaar = 2019

### Inlezen buurtcodes en buurtnamen
Bronbestand CBS van peiljaar inlezen (in dit geval 2019)

In [39]:
# inladen master bestand buurtcodes
buurten = pd.read_csv(source_path+'buurt'+str(peiljaar)+'.csv', sep=';')
# hernoemen kolommen
buurten.columns=['buurtcode', 'buurtnaam']
# kolom maken met buurtcodeLetters ten behoeve van mergen met andere bronnen
buurten['buurtcodeLetters'] = 'BU' + buurten['buurtcode'].astype(str).str.zfill(8)

### Inlezen wijkcodes en wijknamen

In [40]:
wijken = pd.read_csv(source_path+'wijk'+str(peiljaar)+'.csv', sep=';')
# hernoemen kolommen
wijken.columns=['wijkcode', 'wijknaam']

### Inlezen gemeentecodes en gemeentenamen

In [41]:
# inladen master bestand gemeenten
gemeenten = pd.read_csv(source_path+'gem'+str(peiljaar)+'.csv', sep=';')
# hernoemen kolommen
gemeenten.columns=['gemeentecode', 'gemeentenaam']

### Wijkcodes, wijknamen, gemeentecodes, gemeentenamen toevoegen aan buurten<br>En gemeentecodes, gemeentenamen toevoegen aan wijken
Dit is gebaseerd op uitgangspunten: <br>
1. Een wijkcode is af te leiden uit de buurtcodes door de laatste twee cijfers van de buurtcode te verwijderen <br>
2. Een gemeentecode is af te leiden uit de wijkcode door de laatste twee cijfers van de wijkcode te verwijderen <br>
Dit moet altijd kloppen ook na herindelingen. Dit is gecheckt met koppeltabel (adressenbestand). 

In [42]:
buurten['wijkcode'] = buurten['buurtcode'].astype(str).str[:-2].astype(np.int64)
buurten = pd.merge(buurten, wijken, how='left', on='wijkcode')

buurten['gemeentecode'] = buurten['wijkcode'].astype(str).str[:-2].astype(np.int64)
buurten = pd.merge(buurten, gemeenten, how='left', on='gemeentecode')

wijken['gemeentecode'] = wijken['wijkcode'].astype(str).str[:-2].astype(np.int64)
wijken = pd.merge(wijken, gemeenten, how='left', on='gemeentecode')

### Ophalen vernummeringstabel CBS
Dit is een bestand dat door regstat is aangeleverd. Bevat 549 rijen en is consistent met wijzigingsindicator. Bevat alleen buurtcodes 2019 en hun voorganger. Betreft alleen indelingswijziging =2, dus alleen code gewijzigd, niet grens. 
Deze oude buurtcodes gaan bij het ophalen van andere bronnen helpen om een match te vinden.<br>
Na toevoegen zijn er 3 soorten buurtcodes: 
- buurtcodeLetters: dit zijn buurtcodeLetters volgens 2019, de masterlist afgeleid van CBS buurten 2019
- buurtcodeLetters18 (veelal leeg)
- buurtcodeLettersMatch: buurtcode 19 tenzij buurtcode 18 bekend is, dan die van 2018. Deze wordt gebruikt voor matches bij bronnen die 18 betreffen ipv 19.

In [43]:
vernummerd = pd.read_excel(source_path+'vernum2019, gecheckt, consistent met wijzigingsindicator.xlsx', sheet_name ='vernum2019')

In [44]:
# kolomnamen hernoemen 
vernummerd.columns=['buurtcode18', 'buurtcode']
# kolom maken met buurtcodeLetters ten behoeve van mergen met andere bronnen
vernummerd['buurtcodeLetters18'] = 'BU' + vernummerd['buurtcode18'].astype(str).str.zfill(8)

# toevoegen aan buurten
buurten = pd.merge(buurten, vernummerd, how='left', on='buurtcode')

# nieuwe kolom maken gebaseerd op het al dan niet gevuld zijn van buurtcodeLetters18
buurten['buurtcodeLettersMatch']=np.where(buurten['buurtcodeLetters18'].isnull(), buurten['buurtcodeLetters'], buurten['buurtcodeLetters18'])


### Ophalen regionale kerncijfers van CBS ODATA

In [45]:
# dit tabelnummer 84583 is de versie van 2019. 
kerncijfers = pd.DataFrame(cbsodata.get_data('84583NED'))

### Toevoegen regionale kerncijfers aan buurten. Alleen relevante kolommen. 
Het bronbestand regionale kerncijfers telt ruim 100 kolommen. Hiervan zijn er maar ca 5 nodig.<br>
Daarnaast alleen de rijen gebruiken die buurtinformatie bevatten, dus weggooien alle andere rijen.

In [46]:
# filteren alleen buurt: SoortRegio_2 = Buurt met 5 spaties
kerncijfersBuurt = kerncijfers.loc[kerncijfers['SoortRegio_2'] == 'Buurt     ']

# selecteren relevante kolommen
keepcols  = ['Codering_3', 'AantalInwoners_5', 'Bevolkingsdichtheid_33', 'Omgevingsadressendichtheid_106', 'HuishoudensTotaal_28', 'k_65JaarOfOuder_12']
kerncijfersBuurt = kerncijfersBuurt[keepcols]
# hernoemen kolomnamen
kerncijfersBuurt.columns=['buurtcodeLetters', 'inw', 'bevdh', 'oadh', 'hh', 'inw65' ]

### Introductie bm: buurten Main

In [47]:
# toevoegen kerncijfers aan masterbestand gematcht op buurtcodeLetters
# hier wordt bm geintroduceerd: buurten bestand MAIN. 
bm=pd.merge(buurten, kerncijfersBuurt, how='left', on='buurtcodeLetters')

### Ophalen leefbaarometer
Kolommen selecteren en hernoemen en Matchen.<br>
Opmerkingen:
1. Er zijn buurten die niet meer bestaan in de CBS buurten maar nog wel in de leefbarometer Dat komt door herindelingen. Dit is waarschijnlijk de andere kant van het volgende punt<br>
2. Er bleken  buurten die wel bestaan in CBS buurten maar niet in leefbarometer. Dit is op te lossen door te matchen op buurtcodeMatch, dat betekent dat waar van toepassing, wordt teruggegrepen op buurtcode 18<br>
3. Niet alles is op te lossen door teruggrijpen op buurtcode 18 nl. als buurten ook een andere grenzen hebben gekregen. in dit geval is er geen buurtcode18 in ons masterbestand buurt toegevoegd.

In [48]:
# inlezen leefbaarheidsexcels
leefbaarheid=pd.read_excel(source_path+ 'Score_Buurt.xlsx', sheet_name='data')
dimensiescore=pd.read_excel(source_path+ 'dimensiescore_Buurt (stand).xlsx', sheet_name='data(2)')

In [49]:
# selecteren relevante kolommen score
keepcols =['CODE', 'KL16', 'KL18', 'VKL1618']
leefbaarheid=leefbaarheid[keepcols]

# hernoemen sleutelkolom tbv samenvoegen
leefbaarheid.columns = ['buurtcodeLettersMatch', 'KL16', 'KL18', 'VKL1618']

# selecteren relevante kolommen dimensiescore
keepcols =['GBD', 'RLBRMTR16', 'RLBWON16', 'RLBBEV16', 'RLBVRZ16', 'RLBVEI16', 'RLBFYS16', 
                   'RLBRMTR18', 'RLBWON18', 'RLBBEV18','RLBVRZ18', 'RLBVEI18', 'RLBFYS18']

dimensiescore=dimensiescore[keepcols]
dimensiescore.columns = ['buurtcodeLettersMatch', 'RLBRMTR16', 'RLBWON16', 'RLBBEV16', 'RLBVRZ16', 'RLBVEI16', 'RLBFYS16', 'RLBRMTR18', 'RLBWON18', 'RLBBEV18',
           'RLBVRZ18', 'RLBVEI18', 'RLBFYS18']

In [50]:
# samenvoegen
bm=pd.merge(bm, leefbaarheid, how='left', on='buurtcodeLettersMatch')

# toevoegen dimensiescore info
bm=pd.merge(bm, dimensiescore, how ='left', on= 'buurtcodeLettersMatch')

### Ophalen voorzieningen informatie CBS
Deze tabel heet nabijheidsstatistieken en is nodig omdat diverse nabijheidsinformatie in de regionale kerncijfers leeg was. 
N.B. de nabijheidsstatistieken zijn van 2018 en de kerncijfers zijn van 2019. Daarom matchen adhv buurtcodeLettersMatch

In [51]:
nabij = pd.DataFrame(cbsodata.get_data('84463NED'))

### Toevoegen voorzieningen informatie aan buurten
er zijn hier ca 70 interessante indicatoren, hieronder een eerste selectie, wellicht tzt nog eens alle 70 bekijken en
beoordelen op samenhang!

In [52]:
# filteren alleen buurt: SoortRegio_2 = Buurt met 5 spaties
nabij = nabij.loc[nabij['SoortRegio_2'] == 'Buurt     ']

# selecteren relevante kolommen. Totaal zijn er 70 waarvan veel overlap. Dit is een eerste selectie met interessante kandidaten.
keepcols  = ['Codering_3', 'AfstandTotGroteSupermarkt_24', 'Binnen3Km_30', 'AfstandTotHuisartsenpraktijk_5', 'Binnen3Km_38', 
             'Binnen3Km_46', 'Binnen1Km_53', 'AfstandTotZwembad_93', 'Binnen10Km_97', 'Binnen10Km_101', 'Binnen5Km_105']
nabij= nabij[keepcols]

# hernoemen kolomnamen
nabij.columns= ['buurtcodeLettersMatch','AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3', 'AantalRest3', 
                     'AantalKinderopvang1', 'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5']

In [53]:
# toevoegen aan buurtmain obv buurtcodeLettersMatch
bm=pd.merge(bm, nabij, how='left', on='buurtcodeLettersMatch')

### Ophalen ABF info

In [54]:
# inlezen ABF excel met aantallen huishoudens
ink = pd.read_excel(source_path+'Huishoudens in woningen  - Socrates 2019 Midden.xlsx', sheet_name ='Data',na_values = '.')

### Toevoegen ABF info aan buurten
obv buurtcodeLettersMatch omdat buurten van 2018 bleken te zijn

In [55]:
# hernoemen kolommen
ink.columns = ['buurtnaam', 'buurtcode', 'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog']

# creëren buurtcodeLetters
ink['buurtcodeLettersMatch'] ='BU' + ink['buurtcode'].astype(str).str.zfill(8)
# kolommen buurtnaam en buurtcode zijn niet meer nodig en laten vervallen
ink.drop(['buurtnaam', 'buurtcode'], axis=1, inplace=True)

# toevoegen kolom aandeelMH19
ink['aandeelMH19'] = (ink['hh19inkMidden'] + ink['hh19inkHoog']) / (ink['hh19inkLaag'] + ink['hh19inkMidden'] + ink['hh19inkHoog'])

In [56]:
# toevoegen aan bm
bm=pd.merge(bm, ink, how ='left', on= 'buurtcodeLettersMatch')

### Ophalen Calcasa prijs info  (wijk en gemeente)
Huizenprijs ontwikkelingen, is niet beschikbaar op buurtniveau. Daarom op wijkniveau. 
Omdat er vergeleken gaat worden met gemeente niveau moet er ook info op gemeenteniveau worden ingelezen. 

In [57]:
# inlezen Calcasa xlsx wijken. N.b. zo recent mogelijk, maar 2019Q4 nog niet beschikbaar. 
calcWijk = pd.read_excel(source_path + 'calcasa wijken 2019q3.xlsx', na_values='-')\
# verwijderen 1e regel die bevat de datum
calcWijk =  calcWijk.iloc[1:]
# hernoemen kolommen
calcWijk.columns = ['wijknaam', 'gemeentenaam', 'wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2']
# verwijderen de laatste regel die bevat Bron: Calcasa
calcWijk = calcWijk[~calcWijk.wijknaam.str.contains("Calcasa")]

# verwijderen rijen met alleen maar nullen want ca 100 wijken van de 1300 bevatten alleen maar nullen d.i. naN
# Dit zijn vnl bedrijventerreinen.
# door deze verwijdering vervalt ook het dubbel voorkomen van Kraggenburg-bedrijventerrein binnen dezelfde gemeente. 
calcWijk.dropna(axis=0, subset=['wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2'], how='all', inplace=True)

In [58]:
# inlezen Calcasa xlsx gemeenten. N.b. moet zelfde peildatum als wijken hebben 
calcGem = pd.read_excel(source_path + 'calcasa gemeenten 2019q3.xlsx', na_values='-')
# verwijderen 1e regel die bevat de datum
calcGem =  calcGem.iloc[1:]
calcGem.columns = ['gemeentenaam', 'woxGem', 'woxdelta1jrGem', 'woxdelta5jrGem', 'woxwwGem', 'woxm2Gem']
# verwijderen laatste regel die bevat bron: Calcasa
calcGem = calcGem[~calcGem.gemeentenaam.str.contains("Calcasa")]

### Afleiden nieuwe kolommen obv Calcasa info

In [59]:
# toevoegen calGem aan calcWijk
calcWijk=pd.merge(calcWijk, calcGem, how='left', on=['gemeentenaam'])
# afleiden kolom absoluut verschil 5jrs delta wijk en gem, uitkomst is een percentage
calcWijk['woxdelta5jrWGabs'] = calcWijk['woxdelta5jr'] - calcWijk['woxdelta5jrGem']
# afleiden kolom relatief verschil 5jrs delta wijk en gem
calcWijk['woxdelta5jrWGrel'] = calcWijk['woxdelta5jr'] / calcWijk['woxdelta5jrGem']
# afleiden kolom verschil m2 prijs wijk en gem
calcWijk['woxm2WG'] = calcWijk['woxm2'] / calcWijk['woxm2Gem']

### Toevoegen Calcasa info aan wijken

In [60]:
# introducie wm: wijken main
# matchen aan de hand van wijknaam en gemeentenaam. Code niet aanwezig dus dat kan niet. 
# En alleen matchen op wijknaam geeft dubbelen.  Daarom inclusief gemeentenaam. 
wm=pd.merge(wijken, calcWijk, how='left', on=['wijknaam', 'gemeentenaam'])

### Woningvoorraad en bouwjaren Calcasa op buurtniveau

In [61]:
# functie declareren die alle deelbestanden inleest en samenvoegt
def prov():
    # eerst de dfwv opbouwen met een eerste provincie
    provincie='NB'
    prov = pd.read_excel(source_path + 'calcasa wv bouwjaren 2018Q4 ' + provincie + '.xlsx', na_values='-')
    # verwijderen 1e regel die bevat de datum
    prov =  prov.iloc[1:]
    # hernoemen kolommen
    prov.columns = ['buurtnaam', 'gemeentenaam', 'wvtotaal', 'wv<1945', 'wv1945-1970', 'wv1970-1980', 'wv1980-1990', 'wv1990-2000', 'wv2000-2010', 'wv>=2010']
    # verwijderen laatste regel die bevat bron: Calcasa
    prov = prov[~prov.buurtnaam.str.contains("Calcasa")]
    #print(prov.head())
    print('aantal rijen na verwerken van NB: ', len(prov))
    dfwv = prov
    
    # hier de dfwv steeds uitbreiden met een provincie
    for provincie in ['ZE', 'ZH', 'LI', 'OV', 'GE', 'DR', 'FL', 'FR', 'GR', 'NH', 'UT']:
        prov = pd.read_excel(source_path + 'calcasa wv bouwjaren 2018Q4 ' + provincie + '.xlsx', na_values='-')
        # verwijderen 1e regel die bevat de datum
        prov =  prov.iloc[1:]
        # hernoemen kolommen
        prov.columns = ['buurtnaam', 'gemeentenaam', 'wvtotaal', 'wv<1945', 'wv1945-1970', 'wv1970-1980', 'wv1980-1990', 'wv1990-2000', 'wv2000-2010', 'wv>=2010']
        # verwijderen laatste regel die bevat bron: Calcasa
        prov = prov[~prov.buurtnaam.str.contains("Calcasa")]
        #print(prov.head())
        # toevoegen aan main
        dfwv = dfwv.append(prov, ignore_index=True)
        print('aantal rijen na verwerken van', provincie, ': ', len(dfwv))
    return dfwv

In [62]:
#aanroepen van de functie
dfwv=prov()

aantal rijen na verwerken van NB:  1090
aantal rijen na verwerken van ZE :  1114
aantal rijen na verwerken van ZH :  2881
aantal rijen na verwerken van LI :  3164
aantal rijen na verwerken van OV :  3559
aantal rijen na verwerken van GE :  4229
aantal rijen na verwerken van DR :  4467
aantal rijen na verwerken van FL :  4772
aantal rijen na verwerken van FR :  4902
aantal rijen na verwerken van GR :  5014
aantal rijen na verwerken van NH :  6419
aantal rijen na verwerken van UT :  7284


In [63]:
# merge sleutel maken. Hier wordt gematcht op combi gemeentenaam en buurtnaam, omdat Calcasa geen codes levert. 
# Als de code veranderd is van 18 op 19 is de buurtnaam gelijk gebleven. 
dfwv['sleutel'] = dfwv['gemeentenaam']+dfwv['buurtnaam']
bm['sleutel'] = bm['gemeentenaam']+bm['buurtnaam']

In [64]:
# kolommen toevoegen voor de procentuele aandelen van elke bouwperiode
# ivm delen door NA eerst alle NA vullen met 0
dfwv.fillna(0, inplace = True) 

dfwv['aand<1945'] = dfwv['wv<1945'] / dfwv['wvtotaal']
dfwv['aand1945-1970'] = dfwv['wv1945-1970'] / dfwv['wvtotaal']
dfwv['aand1970-1980'] = dfwv['wv1970-1980'] / dfwv['wvtotaal']
dfwv['aand1980-1990'] = dfwv['wv1980-1990'] / dfwv['wvtotaal']
dfwv['aand1990-2000'] = dfwv['wv1990-2000'] / dfwv['wvtotaal']
dfwv['aand2000-2010'] = dfwv['wv2000-2010'] / dfwv['wvtotaal']
dfwv['aand>=2010'] = dfwv['wv>=2010'] / dfwv['wvtotaal']

# verwijderen kolommen buurtnaam en gemeentenaam uit dfwv anders krijg je bij de merge onnodige dubbelen
dfwv.drop(['gemeentenaam', 'buurtnaam'], axis=1, inplace=True)

In [65]:
# toevoegen wv info aan bm
bm = pd.merge(bm, dfwv, how='left', on=['sleutel'])

,buurtcode,buurtnaam,buurtcodeLetters,wijkcode,wijknaam,gemeentecode,gemeentenaam,buurtcode18,buurtcodeLetters18,buurtcodeLettersMatch,...,wv1980-1990,wv1990-2000,wv2000-2010,wv>=2010,aand1945-1970,aand1970-1980,aand1980-1990,aand1990-2000,aand2000-2010,aand>=2010
0,30000,Appingedam-Centrum,BU00030000,300,Wijk 00,3,Appingedam,NaN,NaN,BU00030000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30001,Appingedam-West,BU00030001,300,Wijk 00,3,Appingedam,NaN,NaN,BU00030001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30002,Appingedam-Oost,BU00030002,300,Wijk 00,3,Appingedam,NaN,NaN,BU00030002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30007,Verspreide huizen Damsterdiep en Eemskanaal,BU00030007,300,Wijk 00,3,Appingedam,NaN,NaN,BU00030007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30008,Verspreide huizen ten zuiden van Eemskanaal,BU00030008,300,Wijk 00,3,Appingedam,NaN,NaN,BU00030008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### TO DO. dubbelen verwijderen wv. 30 rijen. 

### Ophalen marktpotentie

In [66]:
# de marktpotentie van 2019 inlezen, deze is al handmatig aangepast voor gemeentes per 1.1.2019 en mp omschrijving is korte variant
mp = pd.read_excel(source_path+ 'MP2019.xlsx', sheet_name='master')
# hernoemen kolommen
mp.columns=['gemeentecodeLetter', 'gemeentenaam', 'mp']
# 1 kolom is niet meer nodig
mp.drop('gemeentecodeLetter', axis=1, inplace=True)

### Toevoegen marktpotentie aan gemeenten

In [67]:
# toevoegen mp aan masterbestand aan de hand van match op gemeentenaam
# introductie gm: gemeenten main
gm = pd.merge(gemeenten, mp, how='left', on='gemeentenaam')

### Ophalen bereikbaarheidsinformatie: mob (van MobilityLabel)

In [68]:
# inlezen mobility label data in xlsx
mob = pd.read_excel(source_path + 'bereikbaarheidsrangen_pc5_incl banen.xlsx', sheet_name ='PC5')

### Omrekenen bereikbaarheidsinformatie mob van pc5 naar buurt
Hiervoor is het adressenbestand nodig. Omdat een pc5 in meerdere buurten voor kan komen moet er een omrekening plaatsvinden.
Er wordt gekozen voor simpelweg middelen. 

In [69]:
adressen = pd.read_csv(source_path+'pc6hnr20190801_gwb.csv', sep=';')

# alle uniek voorkomende combinaties van pc5 en buurtcode creëen vanuit het adressenbestand
adressen['pc5']=adressen['PC6'].str.slice(stop=5)
keepcols =['pc5', 'Buurt2019']
adressen = adressen[keepcols]
adressen.drop_duplicates(inplace=True)

# mob: alleen relevante kolommen
keepcols = ['rekenpunt', 'banen_auto_30min', 'banen_auto_60min', 'banen_ov_30min', 'banen_ov_60min', 'banen_fiets', 
            'anders_reizen_waarde_30min', 'anders_reizen_waarde_60min', 'doorstromingswaarde_30min', 'doorstromingswaarde_60min']
mob=mob[keepcols]

# hernoemen kolommen
mob.columns =['pc5', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'ar30', 'ar60', 'door30', 'door60']

# mob toevoegen aan adressenbestand
adressen=pd.merge(adressen, mob, how='left', on='pc5')

In [70]:
# nu per buurt de gemiddelden berekenen
adressenGroup = adressen.groupby(['Buurt2019']).agg({'auto30': 'mean', 'auto60': 'mean', 'ov30': 'mean', 'ov60': 'mean', 'fiets': 'mean', 
                                   'ar30': 'mean', 'ar60': 'mean', 'door30': 'mean', 'door60': 'mean'})
adressenGroup.reset_index(level=0, inplace=True)
adressenGroup.columns = ['buurtcode', 'auto30', 'auto60','ov30','ov60', 'fiets', 'ar30', 'ar60','door30', 'door60']

### Toevoegen bereikbaarheidsinformatie mob aan buurten

In [71]:
bm=pd.merge(bm, adressenGroup, how='left', on='buurtcode')

### Opslaan resultaten als csv
Encoding=utf-16 zorgt ervoor dat diakritische tekens in Excel goed leesbaar zijn.

In [72]:
# drie basisbestanden
bm.to_csv(save_path+'bm.csv', encoding='utf-16', index=False)
wm.to_csv(save_path+'wm.csv', encoding='utf-16', index=False)
gm.to_csv(save_path+'gm.csv', encoding='utf-16', index=False)

In [73]:
# samengestelde csv. om dubbele kolommen te voorkomen eerst droppen bepaalde kolommen
wm.drop(columns=['wijknaam', 'gemeentecode', 'gemeentenaam'], inplace=True)
gm.drop(columns=['gemeentenaam'], inplace=True)

buurten=pd.merge(bm, wm, how='left', on ='wijkcode')
buurten=pd.merge(buurten, gm, how='left', on ='gemeentecode')

buurten.to_csv(save_path+'buurten.csv', encoding='utf-16', index=False)

### Einde Buurtscores Datamodel. Vervolg: Buurtscores EDA